In [ ]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their question on the files they upload.",
#     model="gpt-4-1106-preview",
#     tools=[{"type": "retrieval"}],
# )
assistant_id = "asst_8g39VCPq4Zcq8MxvNh9SepF6"

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file",
        }
    ]
)
thread

In [ ]:
file = client.files.create(
    file=client.file_from_path("./files/chapter_one.txt"), purpose="assistants"
)
file

In [ ]:
client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="", file_ids=[file.id]
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

In [52]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [ ]:
get_run(run.id, thread.id).status

In [53]:
get_messages(thread.id)

user: I want you to help me with this file
user: 
assistant: I noticed you uploaded a file, but you haven't specified what kind of help you need with it. Could you please clarify what you're looking to do? Are there specific parts of the file you need information on, or do you require a general overview? Any details you can provide will help me assist you better!
user: I want to know where does Winston live and how his apartment looks like.
assistant: Winston lives in Victory Mansions, and his apartment has a very distinct and somewhat oppressive atmosphere. As you enter Victory Mansions, you are greeted with the smell of boiled cabbage and old rag mats. The hallway features a large coloured poster of a man with a heavy black moustache under the caption "BIG BROTHER IS WATCHING YOU". No lift is operational in the building; instead, the flat is located seven flights up, requiring Winston to take the stairs【12†source】.

Inside the flat, there is a telescreen that cannot be completely shu

In [ ]:
send_message(
    thread.id,
    "Where does he work?",
)